# Testing Easy Torch

In [1]:
import sys

In [2]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # View the path and verify

['../', '/Users/federicosiciliano/Desktop/Py_utils/easy_lightning_development/easy_torch/ntb', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/federicosiciliano/Library/Python/3.11/lib/python/site-packages', '/opt/homebrew/lib/python3.11/site-packages', '/opt/homebrew/lib/python3.11/site-packages/Easy_Lightning-0.0.1-py3.11.egg']


In [3]:
import easy_data, easy_exp, easy_torch

## Configuration loading

In [4]:
cfg = easy_exp.cfg.load_configuration("config_nn")

In [5]:
cfg

{'__exp__': {'name': 'prova',
  'project_folder': '../',
  'key_len': 16,
  'key_prefix': '',
  '__imports__': ['torchvision'],
  '__nosave__': {'model.loader_params.num_workers': None,
   'model.trainer_params.accelerator': None,
   'model.trainer_params.enable_checkpointing': None,
   'model.trainer_params.logger': None,
   'model.trainer_params.callbacks.1.ModelCheckpoint.dirpath': None,
   'model.trainer_params.callbacks.1.ModelCheckpoint.filename': None},
  '__sweep__': {'parameters': {}}},
 'data': {'name': 'MNIST',
  'source': 'torchvision',
  'merge_before_split': False,
  'split_keys': {'train_x': ['train_x', 'val_x'],
   'train_y': ['train_y', 'val_y']},
  'train_sizes': [100, 100],
  'test_sizes': [0.2],
  'split_random_state': 21094,
  'one_hot_encode': True,
  'scaling': 'MinMax'},
 'model': {'name': 'resnet18',
  'torchvision_params': {'weights': None},
  'optimizer': {'name': 'Adam', 'params': {'lr': 0.1, 'weight_decay': 0.0005}},
  'log_params': {'on_epoch': True, 'on_s

In [6]:
cfg["data"]

{'name': 'MNIST',
 'source': 'torchvision',
 'merge_before_split': False,
 'split_keys': {'train_x': ['train_x', 'val_x'],
  'train_y': ['train_y', 'val_y']},
 'train_sizes': [100, 100],
 'test_sizes': [0.2],
 'split_random_state': 21094,
 'one_hot_encode': True,
 'scaling': 'MinMax'}

In [7]:
data, _ = easy_data.data.load_data(**cfg["data"])

In [8]:
loaders = easy_torch.preparation.prepare_data_loaders(data, **cfg["model"]["loader_params"])

/Users/federicosiciliano/Desktop/Py_utils/easy_lightning_development/easy_torch/ntb/../easy_torch/preparation.py:57: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [9]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

In [10]:
main_module = easy_torch.model.get_torchvision_model(**cfg["model"])

In [11]:
exp_found, experiment_id = easy_exp.exp.get_experiment_id(cfg)

In [12]:
print(exp_found, experiment_id)

False mDDGQT4noVu41PIU


In [13]:
# Set experiment_id in trainer_params
trainer_params = easy_torch.preparation.prepare_experiment_id(cfg["model"]["trainer_params"], experiment_id)

In [14]:
trainer_params["callbacks"] = easy_torch.preparation.prepare_callbacks(trainer_params)
trainer_params["logger"] = easy_torch.preparation.prepare_logger(trainer_params)
trainer = easy_torch.preparation.prepare_trainer(**trainer_params)

# callbacks = easy_torch.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
# logger = easy_torch.preparation.prepare_logger(cfg["model"]["trainer_params"])
# already_defined = ["callbacks","logger"]
# trainer = easy_torch.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks, logger=logger)

Seed set to 42
Seed set to 42
Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [15]:
loss = easy_torch.preparation.prepare_loss(cfg["model"]["loss"])

Seed set to 42


In [16]:
optimizer = easy_torch.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

Seed set to 42


In [17]:
model = easy_torch.process.create_model(main_module, loss=loss, optimizer=optimizer)

Seed set to 42


In [18]:
easy_torch.process.train_model(trainer, model, loaders)

Seed set to 42

  | Name        | Type             | Params | Mode 
---------------------------------------------------------
0 | main_module | ResNet           | 11.2 M | train
1 | loss        | CrossEntropyLoss | 0      | train
---------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/homebrew/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/homebrew/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [19]:
easy_torch.process.test_model(trainer, model, loaders)

Seed set to 42
/opt/homebrew/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           -1000098704130048.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [20]:
easy_exp.exp.save_experiment(cfg)